In [308]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import re
import requests
import json
from functools import reduce
import hashlib
import numpy as np

## API Data
----------------------

In [343]:
# this grabs json data from website, prints status code, and adds each provider to a dataframe
def grab_api_data():
    response = requests.get("https://bw-interviews.herokuapp.com/data/providers")
    
    try:
        response.raise_for_status()
    except requests.exceptions.HTTPError as e:
        return str(e)

    json_data = response.json()

    json_df = pd.DataFrame()
    for key, value in json_data.items():
        for provider in value:
            json_df = json_df.append(provider, ignore_index=True)

    return json_df
 
api_df = grab_api_data()

## Scrape data from naccrrapps
-------------------------


In [344]:
# this method determines how many pages we need to scrape by extracting the number from the last page url
# it also tells us what are table headers are called

def return_last_page_num_and_headers(soup_obj):
    page = requests.get("http://naccrrapps.naccrra.org/navy/directory/programs.php?program=omcc&state=CA&pagenum=1")
    
    try:
        page.raise_for_status()
    except requests.exceptions.HTTPError as e:
        return str(e)
    
    page.status_code
    soup = bs(page.content, 'html5lib')

    for a in soup_obj.find_all('a', href=True):
        if "last page" in a.text.lower():
            last_page = re.findall('\d+', str(a))[0]
            
    col_list = []
    for column_name in soup.find_all('th'):
        col_list.append(column_name.text)

    return int(last_page),col_list

num_pages , header_names = return_last_page_num_and_headers(soup)

# this scrapes all data from website and puts result in a dataframe.

def grab_all_data(num_pages,col_names):
    table_data_list = []
    for page_number in range(1,num_pages +1):
        page = requests.get("http://naccrrapps.naccrra.org/navy/directory/programs.php?program=omcc&state=CA&pagenum={}".format(page_number))
        soup = bs(page.content, 'html5lib')

        table = soup.find('table', { "class" : "gray sortable" })
        table_rows = table.find_all('tr')

        for rows in table_rows[1:]:
            one_row_of_data = rows.find_all('td') 
            each_row = []
            for element in one_row_of_data:
                each_row.append(element.text)
            table_data_list.append(each_row)
    result_df = pd.DataFrame.from_records(table_data_list,columns=col_names)   
    
    return result_df

scraped_df = grab_all_data(num_pages,header_names)
scraped_df.head()

,Provider Name,Type Of Care,Address,City,State,Zip,Phone,Email
0,La Rue Park Child Development Center,Child Care Center,50 Atrium Way,Davis,CA,95616,(530) 753-8716,snfaria@ucdavis.edu
1,Carousel Preschool,Child Care Center,8333 Airport Blvd.,Los Angeles,CA,90045,(310) 216-6641,jane.uy@carouselschool.com
2,Azusa Discovery Center,Child Care Center,155 W. Arrow Highway,Azusa,CA,91702,(626) 334-1806,jhdmaddox60@msn.com
3,Little Peoples Corner Preschool/daycare,Child Care Center,3844 Walnut Drive #c,Eureka,CA,95503,(707) 445-0339,lsfloren@yahoo.com
4,Olive Knolls Christian School,Child Care Center,6201 Fruitvale Ave.,Bakersfield,CA,93308,(661) 393-3566,wmayes@okcs.us


## Grab Data From CSV
-------------------------------

In [345]:
column_names = ['provider_name','type_of_care','address','city','state','zip','phone']
csv_df = pd.read_csv('x_ca_omcc_providers.csv',header=None,names = column_names)
csv_df.head()

,provider_name,type_of_care,address,city,state,zip,phone
0,La Rue Park Child Development Center,Child Care Center,50 Atrium Way,Davis,CA,95616,5307538716
1,Carousel Preschool,Child Care Center,8328 Airport Blvd.,Los Angeles,CA,90045,3102166641
2,Adventure Club - Quail Glen,Child Care Center,1250 Canevari Dr,Roseville,CA,95747,9167727529
3,Adventure Club - Coyote Ridge,Child Care Center,1751 Morningstar Dr,Roseville,CA,95747,9167727271
4,Azusa Discovery Center,Child Care Center,155 W. Arrow Highway,Azusa,CA,91702,6263341806


## Clean Up Data and Get Final Table
---------------------------------------------------

In [346]:
print(csv_df.columns)
print(scraped_df.columns)
print(api_df.columns)

#rename columns so they are the same throughout all datasets
scraped_df.rename(columns={'Provider Name': 'provider_name', 'Type Of Care': 'type_of_care', 
                           'Address': 'address', 'City': 'city', 'State': 'state','Zip': 'zip'
                           ,'Phone': 'phone','Email': 'email'}, inplace=True)

#remove non digits from phone number
scraped_df['phone'] = scraped_df['phone'].str.replace(r'\D+', '')
api_df['phone'] = api_df['phone'].str.replace(r'\D+', '')

# convert all data to lowercase so we dont run into different capitalization issues 
# this may not be the best choice for our data warehouse but for this task I am ok with it
api_df = api_df.apply(lambda x: x.astype(str).str.lower())
scraped_df = scraped_df.apply(lambda x: x.astype(str).str.lower())
csv_df = csv_df.apply(lambda x: x.astype(str).str.lower())

# This drops about 41 rows
print(csv_df.shape)
csv_df = csv_df.drop_duplicates()
print(csv_df.shape)

print(scraped_df.shape)
scraped_df = scraped_df.drop_duplicates()
print(scraped_df.shape)

print(api_df.shape)
api_df = api_df.drop_duplicates()
print(api_df.shape)

Index(['provider_name', 'type_of_care', 'address', 'city', 'state', 'zip',
       'phone'],
      dtype='object')
Index(['Provider Name', 'Type Of Care', 'Address', 'City', 'State', 'Zip',
       'Phone', 'Email'],
      dtype='object')
Index(['email', 'id', 'owner_name', 'phone', 'provider_name'], dtype='object')
(1500, 7)
(1459, 7)
(1176, 8)
(1176, 8)
(1500, 5)
(1500, 5)


In [347]:
print(csv_df.dtypes)
print("----------------")
print(scraped_df.dtypes)
print("----------------")
print(api_df.dtypes)

provider_name    object
type_of_care     object
address          object
city             object
state            object
zip              object
phone            object
dtype: object
----------------
provider_name    object
type_of_care     object
address          object
city             object
state            object
zip              object
phone            object
email            object
dtype: object
----------------
email            object
id               object
owner_name       object
phone            object
provider_name    object
dtype: object


In [348]:
# here we can see that none of our phone numbers or names have null values so I will 
# use these columns to make a unique identifier
nan_rows = csv_df[csv_df['phone'].isnull()]
print(len(nan_rows))

nan_rows = api_df[api_df['phone'].isnull()]
print(len(nan_rows))

nan_rows = scraped_df[scraped_df['phone'].isnull()]
print(len(nan_rows))

nan_rows = csv_df[csv_df['provider_name'].isnull()]
print(len(nan_rows))

nan_rows = api_df[api_df['provider_name'].isnull()]
print(len(nan_rows))

nan_rows = scraped_df[scraped_df['provider_name'].isnull()]
print(len(nan_rows))

0
0
0
0
0
0


In [349]:
# change datatypes and then use provider name and phone as a unique identifier and then use a md5 to see
# what rows are the same accross data sets.

csv_df = csv_df.astype(object)

csv_df['phone'] = csv_df['phone'].astype(str)
csv_df["name_and_provider"]= (csv_df['provider_name']+ csv_df['phone']).str.strip()

api_df["name_and_provider"]= (api_df['provider_name']+api_df['phone']).str.strip()
scraped_df["name_and_provider"]= (scraped_df['provider_name']+scraped_df['phone']).str.strip()

csv_df['md5'] = [hashlib.md5(val.encode('UTF-8')).hexdigest() for val in csv_df['name_and_provider']]
api_df['md5'] = [hashlib.md5(val.encode('UTF-8')).hexdigest() for val in api_df['name_and_provider']]
scraped_df['md5'] = [hashlib.md5(val.encode('UTF-8')).hexdigest() for val in scraped_df['name_and_provider']]

In [350]:
l1 = scraped_df["md5"].to_list()
l2 = api_df["md5"].to_list()
l3 = csv_df["md5"].to_list()
unique=list(set(l1+l2+l3))
unique_hashes_df = pd.DataFrame(unique,columns=["md5"])
unique_hashes_df

,md5
0,85696d2a6e3d05b5af41870c8a264d7e
1,95da0b1df5493b1cb41d77c8b456259a
2,2bc4a2daf6422d0e36ddb09a31ec12f3
3,87d6eecc4ee295ee5f308c7c5b3d3f19
4,8c020c39173b57b37d6c705898ab13ba
...,...
1616,4f50177c4d710b6c74d68c0f8a8d3ce9
1617,ca46ac743e748183f6a5cc1e6942d0e9
1618,0104a6ab40f3e926c1f6cf3d49f258b0
1619,5f1f164aa58909e68038895dec0e7393


In [351]:
api_df
# email_api id_api owner_name_api phone_api provider_name_api

,email,id,owner_name,phone,provider_name,name_and_provider,md5
0,snfaria@ucdavis.edu,prvdrs_xtvzpytrizsrbvjojkuhkrh6,elizabeth wall,5307538716,la rue park child development center,la rue park child development center5307538716,7ed02181a0aee4d90c3da460fa1d28a7
1,jane.uy@carouselschool.com,prvdrs_9xynilfffjgejjwbjp2uvlje,shaneka kittrell,3102166641,carousel preschool,carousel preschool3102166641,5a4365edeadebc934658fed59018f619
2,jrutledge@roseville.ca.us,prvdrs_6fwq7dwgbvcthrzwbpbuzhwj,dewayne bunning,9167727529,adventure club - quail glen,adventure club - quail glen9167727529,5b981d5d9c85aef8587f91624ad7939c
3,vsisneroz@roseville.ca.us,prvdrs_8yf3tyi0eeqljm1uieozvx68,ernest edmondson,9167727271,adventure club - coyote ridge,adventure club - coyote ridge9167727271,1be14404fdbe9328125f86dde1dd8010
4,jhdmaddox60@msn.com,prvdrs_nh2ozyh77wa1vs2axsmqfnlx,william bliss,6263341806,azusa discovery center,azusa discovery center6263341806,fcedfdc338815a436750bd1eadd2b5c6
...,...,...,...,...,...,...,...
1495,subsidy@kc-education.com,prvdrs_6nfaoq6btqpv56fnskh0rsff,allen harder,5038721464,kce champions llc @ mason,kce champions llc @ mason5038721464,c0ae66b65f73631a20fc864daf01bb69
1496,orcuttlee@yahoo.com,prvdrs_kdnjgmvmabwy13butzqyf6vy,jason moons,8052648234,amy lee ulloa,amy lee ulloa8052648234,66981173d169a10e587146c420b4f437
1497,chapmanmontessori@gmail.com,prvdrs_pdziffwzski97evhotqqw1lr,ana drake,7145915927,chapman montessori school,chapman montessori school7145915927,d80e03797b8e0a793268bae99cef9597
1498,christena@sdkidventures.com,prvdrs_akzhqvph9oyodwo6q7fxlbfb,jill stokes,6195739625,kid ventures liberty station preschool,kid ventures liberty station preschool6195739625,59b1342cfa9eb4a7316997198f4b8ada


,email_api,"id_api,",owner_name_api,phone_api,provider_name_api,name_and_provider,md5
0,snfaria@ucdavis.edu,prvdrs_xtvzpytrizsrbvjojkuhkrh6,elizabeth wall,5307538716,la rue park child development center,la rue park child development center5307538716,7ed02181a0aee4d90c3da460fa1d28a7
1,jane.uy@carouselschool.com,prvdrs_9xynilfffjgejjwbjp2uvlje,shaneka kittrell,3102166641,carousel preschool,carousel preschool3102166641,5a4365edeadebc934658fed59018f619
2,jrutledge@roseville.ca.us,prvdrs_6fwq7dwgbvcthrzwbpbuzhwj,dewayne bunning,9167727529,adventure club - quail glen,adventure club - quail glen9167727529,5b981d5d9c85aef8587f91624ad7939c
3,vsisneroz@roseville.ca.us,prvdrs_8yf3tyi0eeqljm1uieozvx68,ernest edmondson,9167727271,adventure club - coyote ridge,adventure club - coyote ridge9167727271,1be14404fdbe9328125f86dde1dd8010
4,jhdmaddox60@msn.com,prvdrs_nh2ozyh77wa1vs2axsmqfnlx,william bliss,6263341806,azusa discovery center,azusa discovery center6263341806,fcedfdc338815a436750bd1eadd2b5c6
...,...,...,...,...,...,...,...
1495,subsidy@kc-education.com,prvdrs_6nfaoq6btqpv56fnskh0rsff,allen harder,5038721464,kce champions llc @ mason,kce champions llc @ mason5038721464,c0ae66b65f73631a20fc864daf01bb69
1496,orcuttlee@yahoo.com,prvdrs_kdnjgmvmabwy13butzqyf6vy,jason moons,8052648234,amy lee ulloa,amy lee ulloa8052648234,66981173d169a10e587146c420b4f437
1497,chapmanmontessori@gmail.com,prvdrs_pdziffwzski97evhotqqw1lr,ana drake,7145915927,chapman montessori school,chapman montessori school7145915927,d80e03797b8e0a793268bae99cef9597
1498,christena@sdkidventures.com,prvdrs_akzhqvph9oyodwo6q7fxlbfb,jill stokes,6195739625,kid ventures liberty station preschool,kid ventures liberty station preschool6195739625,59b1342cfa9eb4a7316997198f4b8ada


In [352]:
#rename columns so we know what datasource they are coming from

scraped_df.rename(columns={'provider_name': 'provider_name_website', 'type_of_care': 'type_of_care_website', 
                           'address': 'address_website', 'city': 'city_website', 'state': 'state_website',
                           'zip': 'zip_website','phone': 'phone_website','email': 'email_website'}, inplace=True)

csv_df.rename(columns={'provider_name': 'provider_name_csv', 'type_of_care': 'type_of_care_csv', 
                           'address': 'address_csv', 'city': 'city_csv', 'state': 'state_csv',
                           'zip': 'zip_csv','phone': 'phone_csv'}, inplace=True)

api_df.rename(columns={'provider_name': 'provider_name_api','owner_name': 'owner_name_api','id':'id_api'
                       ,'phone': 'phone_api','email': 'email_api'}, inplace=True)

In [353]:
# Specify what colums we want from each data source to bring into our final table.
scraped_columns_to_grab = ['provider_name_website', 'type_of_care_website', 'address_website',
                           'city_website', 'state_website', 'zip_website', 'phone_website',
                           'email_website','md5']
csv_columns_to_grab = ['md5','provider_name_csv', 'type_of_care_csv', 'address_csv', 'city_csv','state_csv', 'zip_csv', 'phone_csv']
api_columns_to_grab = ['md5','email_api', 'id_api', 'owner_name_api', 'phone_api','provider_name_api']

In [371]:
'''Start with base table which is unique md5 hash ids and left join info to it
I decided to keep the columns from each dataset and indicate where they are coming from
this will allow us to COALESCE on columns in our data warehouse in case some are missing.
We can do further cleaning but for the sake of the time limit on this assignment I will continue onto
analysis.
-------------------------------------------------------------------------------------------------------
'''


table_with_site_data = pd.merge(unique_hashes_df,scraped_df[scraped_columns_to_grab],on="md5",how="left")
table_with_csv_data = pd.merge(table_with_site_data,csv_df[csv_columns_to_grab],on="md5",how="left")
all_data_df = pd.merge(table_with_csv_data,api_df[api_columns_to_grab],on="md5",how="left")


#apply the equivalent of a coalesce on columns so we merge until we dont get a null value.
all_data_df["provider_name_final"] = all_data_df['provider_name_website'].combine_first(all_data_df['provider_name_csv'])
all_data_df["type_of_provider_final"] = all_data_df['type_of_care_website'].combine_first(all_data_df['type_of_care_csv'])
all_data_df["address_final"] = all_data_df['address_website'].combine_first(all_data_df['address_csv'])
all_data_df["city_final"] = all_data_df['city_website'].combine_first(all_data_df['city_csv'])
all_data_df["state_final"] = all_data_df['state_website'].combine_first(all_data_df['state_csv'])
all_data_df["zip_final"] = all_data_df['zip_website'].combine_first(all_data_df['zip_csv'])
all_data_df["email_final"] = all_data_df['email_website'].combine_first(all_data_df['email_api'])
all_data_df["zip_final"] = all_data_df['zip_website'].combine_first(all_data_df['zip_csv'])
all_data_df["id_final"] = all_data_df['id_api']
all_data_df["owner_name_final"] = all_data_df['owner_name_api']
all_data_df["phone_final"] = all_data_df['phone_website'].combine_first(all_data_df['phone_csv']).combine_first(all_data_df['phone_api'])




# send final columns we want and result to a csv this will be our table we do analysis on
final_cols_to_grab = ['md5','provider_name_final', 'type_of_provider_final',
                      'address_final', 'city_final', 'state_final', 'zip_final',
                      'email_final', 'id_final', 'owner_name_final', 'phone_final']


final_table = all_data_df[final_cols_to_grab]
final_table.to_csv("final_table.csv", index = False)

In [372]:
final_table

,md5,provider_name_final,type_of_provider_final,address_final,city_final,state_final,zip_final,email_final,id_final,owner_name_final,phone_final
0,85696d2a6e3d05b5af41870c8a264d7e,carlsbad educational foundation- kelly elem.,child care center,4885 kelly dr.,carlsbad,ca,92008,melisa@carlsbaded.org,prvdrs_miyi6txwcnvoqietcwm3whls,donald taylor,7603315566
1,95da0b1df5493b1cb41d77c8b456259a,teesha miller,family child care home,n/a,carmichael,ca,95608,mooremiller@comcast.net,prvdrs_xiqqviitlrxfg39evo44c6q2,eugene alli,9169440069
2,2bc4a2daf6422d0e36ddb09a31ec12f3,linton family child care,family child care home,nan,ramona,ca,92065,temshaa@aol.com,prvdrs_upua5anubsyaqxfijiyer1fp,claudia saba,7607897047
3,87d6eecc4ee295ee5f308c7c5b3d3f19,child development services at country hills el...,child care center,151 n associated rd,brea,ca,92821,kfarrer@bousd.us,prvdrs_lxahbmjoymvf2sdnjmoztczv,robert murphy,7149907527
4,8c020c39173b57b37d6c705898ab13ba,avan family child care,family child care home,n/a,chula vista,ca,91910,avannoory@yahoo.com,prvdrs_sbftcancewekntewjoygxyz3,james trentman,6199479190
...,...,...,...,...,...,...,...,...,...,...,...
1689,4f50177c4d710b6c74d68c0f8a8d3ce9,learning lab preschool,child care center,5000 barranca pkwy,irvine,ca,92604,info@learninglabpreschool.com,prvdrs_mqwkrhq34uupg1il1cqbwcc8,jamie trombley,9493393511
1690,ca46ac743e748183f6a5cc1e6942d0e9,happy tots montessori school & infant center,child care center,1518 pacific coast hwy,harbor city,ca,90710,happytotsmontessori@yahoo.com,prvdrs_xbqysjfkpozinrtpmfolgofx,tony moore,3108916080
1691,0104a6ab40f3e926c1f6cf3d49f258b0,4th r - hollywood park,child care center,4915 harte way,sacramento,ca,95822,hpark4thr@cityofsacramento.org,prvdrs_tlyugnzrqi8hlyk1qmz2xs8j,sandy campos,9162776186
1692,5f1f164aa58909e68038895dec0e7393,santa margarita children's center,child care center,1055 las ovejas ave suite #5,san rafael,ca,94903,office@smccenter.us,prvdrs_jjo01knfov4rs0tf1hzjjqms,derrick martinez,4154991277


## SQL Analysis and Pandas
---------------
#### How many Family Child Care Home providers are there in the dataset?

###### Answer: 627

In [378]:
'''
SELECT 
    COUNT(*) 
FROM 
    final_table 
WHERE 
    type_of_provider_final = 'family child care home'
'''
final_table['type_of_provider_final'].value_counts()

child care center         987
family child care home    627
Name: type_of_provider_final, dtype: int64

#### Which Zip code has the most providers?
###### Answer: 92154

In [379]:
'''
SELECT 
    zip_final,
    COUNT(*) 
FROM 
    final_table 
WHERE 
    type_of_provider_final = 'family child care home'
GROUP BY zip_final
ORDER by zip_final desc
'''

#this gets us just our data with family child care homes
family_child_care_df = final_table[final_table['type_of_provider_final']=='family child care home']

family_child_care_df.groupby('zip_final')['md5'].count().sort_values(ascending= False)

# final_table['type_of_provider_final'].value_counts()

zip_final
92154    25
91913    19
92124    17
92563    16
92114    16
         ..
93436     1
93311     1
93291     1
93245     1
90003     1
Name: md5, Length: 258, dtype: int64